#**Job Description Apps**
 In this notebook, we are going to create Job Description in different ways. 
 - Without any context 
 - Using company URL 
 - Using reference URL 
 ###**Install Dependencies**

In [1]:
!pip install openai gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.6 MB/s eta 0:00:00


###**Retrive API key from Secrets and Set as an ENV**

In [3]:
# Retrieve the API key from Colab's secrets
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [4]:
# Set OPENAI_API_KEY as an ENV
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [9]:
from openai import OpenAI
import gradio as gr

In [10]:
client=OpenAI()

####**App 1: Create Job Description without context**

In [11]:
# Function to generate job description
def generate_job_description(job_title, role_details):
    prompt = f"Create a job description for the following role: {job_title}. Responsibilities: {role_details}. Provide a comprehensive description with key responsibilities, required skills, and qualifications."

    # Create chat completion using OpenAI API
    completion = client.chat.completions.create(
        model="gpt-4o-mini",  # Use the specified model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    # Extract and return the generated response
    return completion.choices[0].message.content


In [12]:
# Create Gradio interface for job description generation
iface_job_desc = gr.Interface(
    fn=generate_job_description,
    inputs=[
        gr.Textbox(label="Job Title", placeholder="e.g., Software Engineer"),
        gr.Textbox(label="Role Details", placeholder="e.g., Design and develop software applications")
    ],
    outputs="text",
    live=False,
    title="Job Description Generator"
)

In [13]:
#Launch the app
iface_job_desc.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1f98be37a4e0ccafa6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


####**App 2:Create Job Description using Company URL**

Feature: Generate multiple job roles with responsibilities, required skills, and qualifications etc.



In [14]:
# URL:https://cloudyuga.guru/
# URL:https://omlogistics.co.in/

In [15]:
!pip install requests bs4

In [16]:
import gradio as gr
from openai import OpenAI
import requests
from bs4 import BeautifulSoup


In [17]:
# Function to extract basic information from a company URL
def extract_company_info(url):
    # Fetch the webpage content
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the title or first heading as a company description
        company_name = soup.title.string if soup.title else "Company Name"
        description_tag = soup.find('meta', {'name': 'description'}) or soup.find('meta', {'property': 'og:description'})
        description = description_tag['content'] if description_tag else "No description found."

        return company_name, description
    except Exception as e:
        return "Error", str(e)

In [18]:
# Function to generate a job description based on the company URL
def generate_job_description_from_url(url):
    company_name, description = extract_company_info(url)

    prompt = f"Generate a detailed job description for a company named {company_name} which specializes in {description}. The company is looking for new talent and you should create a list of potential job roles with responsibilities, required skills, and qualifications for each role. Feel free to suggest a variety of roles based on the company's activities and industry."

    # Create chat completion using OpenAI API
    completion = client.chat.completions.create(
        model="gpt-4o-mini",  # Use the specified model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    # Extract and return the generated response
    return completion.choices[0].message.content.strip()


In [19]:
# Create Gradio interface for generating job description based on URL
iface_url = gr.Interface(
    fn=generate_job_description_from_url,
    inputs=gr.Textbox(label="Company URL", placeholder="Enter the company's URL"),
    outputs="text",
    live=False,
    title="Job Description Generator for a Company"
)

In [20]:
iface_url.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2948186e4377d55783.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


####**App 3 : Create Job Description using Reference URL**

Feature: Enter Company name and reference  URL to create a same kind of Job Description



In [21]:
#URL: https://targetjobs.co.uk/careers-advice/job-descriptions/sales-executive-job-description
#URL: https://www.simplilearn.com/devops-engineer-job-description-article

In [22]:
import gradio as gr
from openai import OpenAI
import requests
from bs4 import BeautifulSoup

In [23]:
# Function to extract job description from a given URL
def extract_job_description(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the main content (job description)
        paragraphs = soup.find_all("p")  # Common tag for job descriptions
        job_text = "\n".join([p.get_text() for p in paragraphs if p.get_text()])

        return job_text if job_text else "No job description found."

    except Exception as e:
        return str(e)

In [24]:
def generate_job_description(company_name, url):
    job_description = extract_job_description(url)

    if not job_description:
        return "Error fetching job description from the provided URL. Please check the URL."

    system_prompt = f"""
    You are an experienced HR specialist at {company_name}, a well-established company in its industry. Your task is to create a job description that closely follows the content from the provided URL. Please keep the structure and key details as close as possible to the original description.

    Don't use too much creativity or add extra information. Your focus should be on accurately reflecting the job role as it is described in the URL, adjusting it only slightly to match the company’s name and context.

    Below is the job description extracted from another source. Using this, generate a customized job posting for {company_name}, following these instructions:
    ---
    {job_description}
    - Ensure the job description stays as close as possible to the original.
    - Only make necessary adjustments to include the company name, company culture, and industry relevance.
    - Don't add extra details, just make minor adjustments to fit {company_name}.
    """

    # Generate job description using OpenAI API
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "Please generate the job description."}
        ]
    )

    return response.choices[0].message.content.strip()


In [25]:
# Create Gradio app
iface = gr.Interface(
    fn=generate_job_description,
    inputs=[
        gr.Textbox(label="Company Name", placeholder="Enter company name"),
        gr.Textbox(label="Job Description URL", placeholder="Enter the job description URL")
    ],
    outputs="text",
    title="Job Description Generator based on URL",
    live=False
)

In [26]:
# Launch the app
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8531de780add051c93.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
